In [ ]:

# =======================
# 📌 1. Install dependencies
# =======================
!pip install ultralytics deep-sort-realtime mediapipe opencv-python matplotlib pandas --quiet


In [ ]:

# =======================
# 📌 2. Unzip dataset (archive.zip)
# =======================
!unzip -q /content/archive.zip -d /content/dataset

import os
dataset_root = "/content/dataset"
print("Dataset contents:", os.listdir(dataset_root))


In [ ]:

# =======================
# 📌 3. Import libraries
# =======================
import cv2
import glob
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import mediapipe as mp
import pandas as pd
import numpy as np

# Load YOLOv8 for person detection
model = YOLO('yolov8n.pt')  # light version for CPU

# DeepSORT tracker
tracker = DeepSort(max_age=30)

# Mediapipe pose model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1)


In [ ]:

# =======================
# 📌 4. Helper: Classify suspicious behavior
# =======================
def classify_suspicious_behavior(keypoints):
    """
    Example simple rule-based suspicious behavior:
    - If person crouches (hip < knee in y-axis)
    - If both hands above head
    """
    if keypoints is None:
        return "normal"
    
    try:
        left_hip = keypoints[mp_pose.PoseLandmark.LEFT_HIP.value]
        left_knee = keypoints[mp_pose.PoseLandmark.LEFT_KNEE.value]
        left_wrist = keypoints[mp_pose.PoseLandmark.LEFT_WRIST.value]
        left_shoulder = keypoints[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

        # Rule 1: crouching
        if left_hip[1] > left_knee[1]:
            return "crouching"

        # Rule 2: hands up
        if left_wrist[1] < left_shoulder[1]:
            return "hands_up"

    except:
        pass
    
    return "normal"


In [ ]:

# =======================
# 📌 5. Process video frames
# =======================
FRAMES_PATH = glob.glob(os.path.join(dataset_root, "*.jpg"))  # adjust if videos
FRAMES_PATH.sort()

output_video_path = "/content/suspicious_output.mp4"

if not FRAMES_PATH:
    raise FileNotFoundError("No frames found. Ensure archive.zip contains .jpg frames.")

sample_frame = cv2.imread(FRAMES_PATH[0])
h, w, _ = sample_frame.shape
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), 30, (w, h))

log_data = []

for idx, frame_path in enumerate(FRAMES_PATH):
    frame = cv2.imread(frame_path)
    results = model(frame, verbose=False)
    
    detections = []
    for r in results:
        for box in r.boxes:
            cls = int(box.cls[0])
            if model.names[cls] == "person":
                x1, y1, x2, y2 = box.xyxy[0]
                conf = float(box.conf[0])
                detections.append(((x1, y1, x2-x1, y2-y1), conf, cls))
    
    tracks = tracker.update_tracks(detections, frame=frame)
    
    for t in tracks:
        if not t.is_confirmed():
            continue
        ltrb = t.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)
        
        person_crop = frame[y1:y2, x1:x2]
        rgb_crop = cv2.cvtColor(person_crop, cv2.COLOR_BGR2RGB)
        result = pose.process(rgb_crop)
        
        if result.pose_landmarks:
            keypoints = [(lm.x, lm.y) for lm in result.pose_landmarks.landmark]
            behavior = classify_suspicious_behavior(keypoints)
        else:
            behavior = "normal"
        
        color = (0,0,255) if behavior != "normal" else (0,255,0)
        cv2.rectangle(frame, (x1,y1), (x2,y2), color, 2)
        cv2.putText(frame, behavior, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        log_data.append({"frame": idx, "id": t.track_id, "behavior": behavior})
    
    out.write(frame)

out.release()

pd.DataFrame(log_data).to_csv("/content/suspicious_log.csv", index=False)
print("Processing complete. Video saved at:", output_video_path)


In [ ]:

# =======================
# 📌 6. View results
# =======================
import matplotlib.pyplot as plt
import pandas as pd

df = pd.read_csv("/content/suspicious_log.csv")
print(df.head())

behavior_counts = df["behavior"].value_counts()
plt.bar(behavior_counts.index, behavior_counts.values, color=['green','red','orange'])
plt.title("Suspicious Behavior Frequency")
plt.show()
